In [12]:
import scipp as sc
import numpy as np
import matplotlib.pyplot as plt
import plopp as pp
%matplotlib widget

## Load vanadium data
number_of_Q_points=16

#Preallocate
intensity_values=np.zeros((number_of_Q_points,1024))
error_values=np.zeros((number_of_Q_points,1024))

# Load data into a matrix
for Q in range(16):
    filename = '../../IN16b_GGG_data/vanadium_Q' +str(Q+1) +'.dat'

    data_array = np.loadtxt(filename)
    energy_values=data_array[:, 0] #should be the same for all Q
    # EnergyValues[Q,:]=data_array[:, 0]
    intensity_values[Q,:]=data_array[:,1]
    error_values[Q,:]=data_array[:,2]

# Define energy, q and intensity as scipp variables with units, and make a DataArraw
energy=sc.array(dims=['energy'],values=energy_values/1000,unit='meV')
Q=sc.array(dims=['Q'],values=range(number_of_Q_points))
intensity=sc.array(dims=['Q','energy'],values=intensity_values,variances=error_values*error_values) #The variance is the square of the uncertainty!

vanadium_data = sc.DataArray(data=intensity, coords={'Q':Q,'energy': energy})

## Bin and plot data 

Q_bins=4
energy_bins=sc.scalar(1e-3*0.2, unit='meV')

intensity_min=0.0
intensity_max=0.06

energy_min = -0.02 * sc.Unit('meV')
energy_max = 0.02 * sc.Unit('meV')

# da.coords['Energy'] = da.coords['Energy'].to(unit='micro*eV') #optional change the scale to mueV
binned_vanadium_data=vanadium_data.flatten(to='dummy').bin(energy=energy_bins,Q=Q_bins).bins.mean() #can add .plot() to plot it

binned_vanadium_data.plot()



InteractiveFig(children=(HBar(), HBox(children=(VBar(children=(Toolbar(children=(ButtonTool(icon='home', layou…

In [11]:

# Slice the binned data
pp.slicer(binned_vanadium_data['energy',energy_min:energy_max],vmin=intensity_min,vmax=intensity_max,
     keep=['energy'],
     linestyle='none',
     marker='o',
     markerfacecolor='none',
     color='k'
)





Box(children=(InteractiveFig(children=(HBar(), HBox(children=(VBar(children=(Toolbar(children=(ButtonTool(icon…

In [ ]:
## Fit vanadium data